# Quasi Majoranas in 3D


In [ ]:
import hpc05

client, dview, lview = hpc05.start_remote_and_connect(100, profile='pbs2',
                                               folder='~/Work/induced_gap_B_field/',
                                               timeout=300, del_old_ipcluster=False)

# client, dview, lview = hpc05.connect.connect_ipcluster(50, profile='pbs2',
#                                                       folder='~/Work/induced_gap_B_field/',
#                                                       timeout=300)


In [ ]:
# 1. Standard library imports
import asyncio
from copy import copy
from functools import partial
from operator import itemgetter

# 2. External package imports
import holoviews as hv
import kwant
import matplotlib.pyplot as plt
import numpy as np

# 3. Internal imports
import funcs
import common
from common import loss

# hv.notebook_extension('matplotlib')
print(kwant.__version__)

import adaptive
adaptive.notebook_extension()

# Wire with intrinsic SC

In [ ]:
syst_pars = dict(a=10, L=3000, r=35, shape='circle', with_leads=True, right_lead=True)

params = dict(alpha=70, B_x=0.5, B_y=0, B_z=0, Delta=0.25, g=50, orbital=True, mu_sc=100,
              mu=10, mu_lead=10, c_tunnel=3/4, **funcs.constants.__dict__)

syst_pars['L_barrier'] = 0
V_barrier_mu = syst_pars['L_barrier']  # Exactly where the superconductor starts
V_barrier_sigma = 500
V_barrier_height = params['mu'] + 5  # Make it larger
params['V'] = lambda x, y, z: funcs.V_barrier(x, V_barrier_height, V_barrier_mu, V_barrier_sigma)

syst = funcs.make_simple_3d_wire(**syst_pars)

kwant.plot(syst);

# Wire with external SC

In [ ]:
syst_pars = dict(a=10, angle=0, onsite_disorder=False,
                 L=3000, coverage_angle=135, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True)

params = dict(alpha=70, B_x=0.5, B_y=0, B_z=0, Delta=60, g=50, orbital=True, mu_sc=100,
              mu=10, mu_lead=10, c_tunnel=3/4, **funcs.constants.__dict__)

syst_pars['L_barrier'] = 700
V_barrier_mu = syst_pars['L_barrier']  # Exactly where the superconductor starts
V_barrier_sigma = 200
V_barrier_height = params['mu'] + 5  # Make it larger
params['V'] = lambda x, y, z: funcs.V_barrier(x, V_barrier_height, V_barrier_mu, V_barrier_sigma)

syst = funcs.make_3d_wire(**syst_pars)

kwant.plot(syst);

# Topological phase diagram

In [ ]:
from functools import partial


intrinsic_sc = True

lead_pars = dict(a=10, angle=0,
                 coverage_angle=135, r1=35, r2=70, shape='circle',
                 with_shell=True, A_correction=True, rotate_spin_orbit=False)

params = dict(alpha=70, B_x=0, B_y=0, B_z=0, Delta=60, g=50, orbital=True,
              mu=10, mu_lead=10, mu_sc=100, c_tunnel=3/4, V=lambda *x: 0,
              **funcs.constants.__dict__)

if intrinsic_sc:
    params['Delta'] = 0.25
    lead_pars['r'] = lead_pars['r1']
    for k in ['coverage_angle', 'angle', 'r1', 'r2', 'A_correction', 'with_shell']:
        lead_pars.pop(k, None)
    lead_pars['superconducting'] = True

def get_pfaffians(x, keys, intrinsic_sc, params=params, lead_pars=lead_pars):
    import collections
    import funcs
    import scipy.sparse.linalg as sla
    if not isinstance(x, collections.Iterable):
        x = [x]
    for key, val in zip(keys, x):
        params[key] = val

    if intrinsic_sc:
        lead = funcs.make_simple_lead(**lead_pars).finalized()
    else:
        lead = funcs.make_lead(**lead_pars).finalized()

    return funcs.calculate_pfaffian(lead, params)

def loss_per_interval(interval, scale, function_values):
    import math
    x_left, x_right = interval
    x_scale, y_scale = scale
    dx = (x_right - x_left) / x_scale
    return dx

# learner = adaptive.Learner1D(partial(get_pfaffians, keys=['B_x']), [0, 2], loss_per_interval)
learner = adaptive.Learner2D(partial(get_pfaffians, keys=['B_x', 'mu_lead'],
                                     intrinsic_sc=intrinsic_sc), ([0, 4], [7, 40]))

In [ ]:
runner = adaptive.Runner(learner, client, goal=lambda l: l.loss() < 0.01)

In [ ]:
runner.task.print_stack(), learner.n

In [ ]:
%%output size=200
learner.plot(tri_alpha=0.)

In [ ]:
learner.loss()

# Wavefunctions in the lead

In [ ]:
lead = syst.leads[0]
funcs.plot_wfs_in_cross_section(lead, params, 0)

# Potential shape

In [ ]:
V = lambda x, y, z: funcs.V_barrier(x, V_barrier_height=params['mu'] + 5,
                                                  V_barrier_mu=syst_pars['L_barrier'],
                                                  V_barrier_sigma=200)

hv.Curve([(x, V(x, 0,0)) for x in range(syst_pars['L'] + syst_pars['L_barrier'])], vdims=['V'])

# Finite sized system spectrum

In [ ]:
params = dict(alpha=70, B_x=0, B_y=0, B_z=0, Delta=60, g=50, orbital=False,
              mu=14, mu_lead=14, mu_sc=100, c_tunnel=3/4,
              **funcs.constants.__dict__)

syst_pars = dict(a=10, angle=0, onsite_disorder=False,
                 L=3000, coverage_angle=135, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True, right_lead=True)

syst_pars['L_barrier'] = 0
V_barrier_mu = syst_pars['L_barrier']  # Exactly where the superconductor starts
V_barrier_sigma = 700
V_barrier_height = params['mu'] + 5  # Make it larger
params['V'] = lambda x, y, z: funcs.V_barrier(x, V_barrier_height, V_barrier_mu, V_barrier_sigma)


intrinsic_sc = True

if intrinsic_sc:
    params['Delta'] = 0.25
    syst_pars['r'] = syst_pars['r1']
    for k in ['coverage_angle', 'angle', 'r1', 'r2',
              'A_correction', 'with_shell', 'onsite_disorder']:
        syst_pars.pop(k, None)

def get_energies(x, key, intrinsic_sc, params=params, syst_pars=syst_pars):
    import funcs
    import scipy.sparse.linalg as sla
    params[key] = x
    
    if intrinsic_sc:
        syst = funcs.make_simple_3d_wire(**syst_pars)
    else:
        syst = funcs.make_3d_wire(**syst_pars)

    params['V'] = lambda x, y, z: funcs.V_barrier(x,
                                                  V_barrier_height=params['height'], 
                                                  V_barrier_mu=syst_pars['L_barrier'],
                                                  V_barrier_sigma=200)

    ham = syst.hamiltonian_submatrix(params=params, sparse=True).tocsc()
    evals, evecs = sla.eigsh(ham, sigma=0, k=30)
    return evals#, evecs


def loss_per_interval(interval, scale, function_values):
    x_left, x_right = interval
    x_scale, _ = scale
    dx = (x_right - x_left) / x_scale
    return dx


def plot(learner):
    Bs = list(learner.data.keys())
    es = np.array([e for e in learner.data.values()])
    return hv.Overlay([hv.Scatter((Bs, e)).opts(style=dict(color='k')) for e in es.T])


def plots(learners, heights):
    return hv.HoloMap({height: plot(l) for l, height in zip(learners, heights)})


heights = [5, 10, 15, 20]
learners = []
for params['height'] in heights:
    f = partial(get_energies, key='B_x', intrinsic_sc=intrinsic_sc, params=copy(params))
    learner = adaptive.Learner1D(f, [0, 4], loss_per_interval)
    learner.cdims = {'height': params['height']}
    learners.append(learner)
learner = adaptive.BalancingLearner(learners)

In [ ]:
runner = adaptive.Runner(learner, client)

In [ ]:
[(l.n, l.loss()) for l in learners]

In [ ]:
%%output size=200
plots(learners, heights).select(y=(-.5, .5))

In [ ]:
runner.task.print_stack(), learner.loss()

# Smallest gap phase diagram


In [ ]:
params['height'] = params['mu'] + 3

In [ ]:
def smallest_gap(x, keys, intrinsic_sc, params=params, syst_pars=syst_pars):
    import funcs
    import numpy as np
    import scipy.sparse.linalg as sla

    for _x, key in zip(x, keys):
        params[key] = _x

    if intrinsic_sc:
        syst = funcs.make_simple_3d_wire(**syst_pars)
    else:
        syst = funcs.make_3d_wire(**syst_pars)

    params['V'] = lambda x, y, z: funcs.V_barrier(x,
                                                  V_barrier_height=params['height'], 
                                                  V_barrier_mu=syst_pars['L_barrier'],
                                                  V_barrier_sigma=500)

    ham = syst.hamiltonian_submatrix(params=params, sparse=True).tocsc()
    evals, _ = sla.eigsh(ham, sigma=0, k=30, which='LM')
    return np.abs(evals).min()

learners = []

f = partial(smallest_gap, keys=['B_x', 'mu'], intrinsic_sc=intrinsic_sc, params=copy(params))
learner = adaptive.Learner2D(f, ([0, 4], [7, 20]), common.loss)

In [ ]:
runner = adaptive.Runner(learner, client)

In [ ]:
learner.n

In [ ]:
%%output size=200
%%opts Image [colorbar=True]
im = learner.plot(tri_alpha=0)
im

# Random crap

In [ ]:
# %%opts Path [aspect='square']
params = dict(c_tunnel=3/4, B_x=0.5, B_y=0, B_z=0, V_barrier=40, g=50, mu_sc=100,
              alpha=70, orbital=False, V=lambda x, y, z: 10 * z / 35, Delta=60,
              **funcs.constants.__dict__)

lead_pars = dict(a=10, angle=0,
                 coverage_angle=135, r1=35, r2=70, shape='circle',
                 with_shell=True, A_correction=True, rotate_spin_orbit=False)

lead = funcs.make_lead(**lead_pars).finalized()
params['mu_lead'] = params['mu_sc'] = 15
params['B_y'] = 0
ks = np.linspace(-1, 1)
Es = funcs.bands(lead, params, ks)
p1 = hv.Path((ks, Es))[:, -100:100]

p1[:, -25:25]# + p1[:, -2:2]

In [ ]:
%%time
gap = funcs.find_gap(lead, params)
print(f'The bandgap is {gap} meV')

In [ ]:
params = dict(alpha=70, B_x=0, B_y=0, B_z=0, Delta=60, g=50, orbital=True,
              mu=15, mu_lead=15, mu_sc=100, c_tunnel=3/4, V=lambda x,y,z:0, V_barrier=50,
              **funcs.constants.__dict__)

S = kwant.smatrix(syst, params=params)

In [ ]:
funcs.andreev_conductance(syst, params, 0)

# Tuning the gap

In [ ]:
params = dict(c_tunnel=3/4, B_x=0, B_y=0, B_z=0, V_barrier=50, g=50,
              alpha=70, orbital=True, V='lambda x, y, z: 10 * z / 35', mu_sc=100,
              **funcs.constants.__dict__)

syst_pars = dict(a=10, angle=0, coverage_angle=135, r1=35, r2=70,
                 shape='circle', with_shell=True, A_correction=True,
                 rotate_spin_orbit=False)

def lowest_energy(x, syst_pars, params):
    import funcs
    import numpy as np
    lead = funcs.make_lead(**syst_pars).finalized()
    params['mu_lead'], params['Delta'] = x
    # Combine the fixed parameters `params` and the changing
    # parameters `val` to one dict and evaluate the string
    # lambda functions.
    params = funcs.parse_params(params)

    # Create the Hamiltonian `ham` at k=0.
    h0 = lead.cell_hamiltonian(params=params)
    t0 = lead.inter_cell_hopping(params=params)
    ham = h0 + t0 + t0.conj().T

    # Find the energies.
    ev = np.linalg.eigvalsh(ham)
    
    # Return a combined dictionary with the results and input.
    return np.abs(ev).min()

def find_crossings(f, g):
    return np.argwhere(np.diff(np.sign(f - g)) != 0).reshape(-1)

def get_Delta(learner, Delta_ind):
    plot = learner.plot(n=1000).redim(x='mu', y='Delta', z='E_gap')
    line = plot.reduce(['mu'], np.min)
    idx = find_crossings(line.data['E_gap'], Delta_ind)[-1]
    Delta = line.data['Delta'][idx]
    return Delta

In [ ]:
learner = adaptive.Learner2D(partial(lowest_energy, syst_pars=syst_pars, params=params),
                             [(5, 20), (0, 200)], loss)

runner = adaptive.Runner(learner, adaptive.Se, goal=lambda l: l.loss() < 0.001)

In [ ]:
runner.task.print_stack()

In [ ]:
%%opts Image [aspect='square']
plot = learner.plot().redim(x='mu', y='Delta', z='E_gap')
plot

In [ ]:
%%opts Scatter (size=10, color='r')
Delta_ind = 0.4

line = plot.reduce(['mu'], np.min)
line * hv.HLine(0.4) * hv.Scatter((get_Delta(learner, Delta_ind), Delta_ind))

In [ ]:
Deltas = np.arange(*learner.bounds[1])
hm = hv.HoloMap({Delta: plot.sample(Delta=Delta)[:, 0:1] for Delta in Deltas})
hm * hv.HLine(0.4)